<div style="text-align: center;">
    <h1>TC's Capstone Project for Applied Data Science</h1>
    <h2>This notebook is being used for my Coursera's Applied Data Science Capstone Project</h2>
    <span><img src="https://upload.wikimedia.org/wikipedia/en/a/aa/Bart_Simpson_200px.png" alt="Bart Simpsons" /></span>
</div>

In [1]:
# 0. Installing Beautiful Soup 4 for scraping html/xml doc
!pip install beautifulsoup4
!pip install lxml
!easy_install -m  lxml


     |████████████████████████████████| 122kB 7.1MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 4.9MB/s eta 0:00:01     |██████████████████████████████▎ | 5.2MB 4.9MB/s eta 0:00:01
Searching for lxml
Best match: lxml 4.5.1

Using /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages
Because this distribution was installed --multi-version, before you can
import modules from this package in an application, you will need to
'import pkg_resources' and then use a 'require()' call similar to one of
these examples, in order to select the desired version:

    pkg_resources.require("lxml")  # latest installed version
    pkg_resources.require("lxml==4.5.1")  # this exact version
    pkg_resources.require("lxml>=4.5.1")  # this version or higher

Processing dependencies for lxml
Finished processing dependencies for lxml


In [2]:
# Install python geocoder
!pip install geocoder

     |████████████████████████████████| 102kB 3.5MB/s ta 0:00:01
     |████████████████████████████████| 92kB 6.6MB/s eta 0:00:011
     |████████████████████████████████| 829kB 11.7MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [3]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

In [4]:
# 1. Import Library
import numpy as np 
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import lxml
import lxml.etree

# import geocoder
import geocoder 

### Remarks:
1. Beautiful Soup library is used to parse the HTML page into BeautifulSoup Object object for result manipulation
2. Default parser has been used in this case

In [5]:
# 2. Get the page on wiki
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(url)
soup = BeautifulSoup(res.content)
raw_tables_contents = soup.table.find_all("tr")

In [6]:
# 3. transforming data from tables_contens into df
# 3.1 Columns
columns_label = raw_tables_contents[0].get_text().replace(" ", "")
columns_label_list = list(filter(None, columns_label.split('\n')))
postal_codes_df = pd.DataFrame(columns=columns_label_list)

In [7]:
# 3.2 Actual data
# 3.2.1 Transform raw data and append into the df
raw_data = raw_tables_contents[1:]
for idx, row in enumerate(raw_data):
   data_row = list(filter(None, row.get_text().split('\n')))
   postal_codes_df.loc[idx] = data_row

In [8]:
# 3.2.2 Cleaning the df
# Remove row with Borough == 'Not assigned'
postal_codes_df = postal_codes_df[postal_codes_df.Borough != 'Not assigned']

### Remarks:
1. In the data table, if Neightborhood is 'Not assigned', Borough must be 'Not assigned' as well, so no extra efforts are spent to clean such data. <br/>
2. Neighborhoods with same PostalCode are combined into one row, delimited by ',' by default, so no extra efforts are spent to clean such data.

In [9]:
postal_codes_df[postal_codes_df.Neighborhood == 'Not assigned'].shape

(0, 3)

In [10]:
postal_codes_df.reset_index(drop=True, inplace=True)
postal_codes_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
postal_codes_df.shape

(103, 3)

In [12]:
# 4.1 Get the csv for Latitude and Longitude
!wget -O Geospatial_data.csv http://cocl.us/Geospatial_data

--2020-06-18 00:29:37--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.194, 158.85.108.83, 158.85.108.86
Connecting to cocl.us (cocl.us)|169.48.113.194|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-06-18 00:29:37--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-06-18 00:29:39--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-06-18 00:29:39--  https://ibm.box.com/publ

In [13]:
geo_partial_data = pd.read_csv("Geospatial_data.csv", delimiter=",", names=['PostalCode', 'Latitude', 'Longitude'])
geo_partial_data = geo_partial_data.iloc[1:].set_index('PostalCode')
# geo_partial_data.reset_index(drop=True)
geo_partial_data.head()

,Latitude,Longitude
PostalCode,,
M1B,43.8066863,-79.1943534
M1C,43.7845351,-79.1604971
M1E,43.7635726,-79.1887115
M1G,43.7709921,-79.2169174
M1H,43.773136,-79.2394761


In [14]:
postal_codes_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
# 4.2 Merge geo_partial_data with postal_codes_df
pc_geo_df = postal_codes_df.join(geo_partial_data, how='inner', on='PostalCode')
pc_geo_df.Latitude = pc_geo_df.Latitude.astype(float)
pc_geo_df.Longitude = pc_geo_df.Longitude.astype(float)
pc_geo_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Starting to explore and cluster the neighborhoods in Toronto
#### 5.1 Narrow down our scope to Toronto specific

In [16]:
# Explore and cluster the neighborhoods in Toronto
# 5.1 Get only the rows of data for Toronto
toronto_df = pc_geo_df[pc_geo_df['Borough'].str.upper().str.contains('TORONTO')]
# toronto_df.set_index('PostalCode')
toronto_df = toronto_df.reset_index(drop=True)
# toronto_df.shape
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### 5.2 Create the map of Toronto and plot all the locations we have from toronto_df

In [17]:
# We have 39 locations to explore and cluste in Toronto
# 5.2 Create map of Toronto
address = 'Toronto'
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
tor_latitude = location.latitude
tor_longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(tor_latitude, tor_longitude))

# create map of Toronto
map_toronot = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronot)
    
map_toronot

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### 5.3 Define Foursquare API credential

In [18]:
# @hidden_cell

CLIENT_ID = '0YEC5AQWOTDYWPW1CMA4ZGYILP4I3RZ2NBQIEDA0AVH1W1QQ' # your Foursquare ID
CLIENT_SECRET = 'TQXY0TLCPZMSVMFWYO5SMBH5UOWVHZ5MOEMELOP2IP3JRHWX' # your Foursquare Secret
VERSION = '20200616' # Foursquare API version

LIMIT = 100

#### 5.4 Explore the locations of toronto_df

####    Create a function to iteratively retrieve the nearby venue for all our neighbours in the toronto_df

In [19]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print("Processing: ", name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
toronto_venue = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Processing:  Regent Park, Harbourfront
Processing:  Queen's Park, Ontario Provincial Government
Processing:  Garden District, Ryerson
Processing:  St. James Town
Processing:  The Beaches
Processing:  Berczy Park
Processing:  Central Bay Street
Processing:  Christie
Processing:  Richmond, Adelaide, King
Processing:  Dufferin, Dovercourt Village
Processing:  Harbourfront East, Union Station, Toronto Islands
Processing:  Little Portugal, Trinity
Processing:  The Danforth West, Riverdale
Processing:  Toronto Dominion Centre, Design Exchange
Processing:  Brockton, Parkdale Village, Exhibition Place
Processing:  India Bazaar, The Beaches West
Processing:  Commerce Court, Victoria Hotel
Processing:  Studio District
Processing:  Lawrence Park
Processing:  Roselawn
Processing:  Davisville North
Processing:  Forest Hill North & West, Forest Hill Road Park
Processing:  High Park, The Junction South
Processing:  North Toronto West,  Lawrence Park
Processing:  The Annex, North Midtown, Yorkville
Pr

In [21]:
print(toronto_venue.shape)
toronto_venue.head()

(1627, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park


In [22]:
toronto_venue.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",19,19,19,19,19,19
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,67,67,67,67,67,67
Christie,17,17,17,17,17,17
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


In [23]:
print('There are {} uniques categories.'.format(len(toronto_venue['Venue Category'].unique())))

There are 240 uniques categories.


#### 5.5 Starting to analyze each neighbour

#### 5.5.1 Transforming venue data from the toronto_venue result

In [24]:
# create empty dataframe with all venue cateogry list as columns
toronto_onehot = pd.get_dummies(toronto_venue[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
# toronto_onehot['Neighborhood'] = toronto_venue['Neighborhood'] 
toronto_onehot.rename(columns={"Neighborhood": "Neighborhood_tbd"}, inplace=True)
toronto_onehot.insert(0, 'Neighborhood', toronto_venue['Neighborhood'])
toronto_onehot.drop(columns=['Neighborhood_tbd'], inplace=True)

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(39, 240)

#### Get the top 5 most popular venues per neighborhood

#### Function to sort venues in descending order

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### 5.5.2 Get the top 10 venues for each neighborhood

In [107]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Restaurant,Cheese Shop,Café,Beer Bar,Japanese Restaurant,Hotel
1,"Brockton, Parkdale Village, Exhibition Place",Café,Performing Arts Venue,Breakfast Spot,Coffee Shop,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Garden Center,Skate Park,Restaurant,Recording Studio,Pizza Place,Park,Garden,Spa
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Boutique,Harbor / Marina,Plane,Coffee Shop,Boat or Ferry,Sculpture Garden,Rental Car Location,Airport Terminal
4,Central Bay Street,Coffee Shop,Italian Restaurant,Japanese Restaurant,Sandwich Place,Café,Salad Place,Dessert Shop,Middle Eastern Restaurant,Thai Restaurant,Department Store


#### 5.5.3 Cluster the neighborhood by using K-mean

In [180]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [181]:
# add clustering labels
neighborhoods_venues_sorted.drop(columns=['Cluster Labels'], inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

toronto_merged.shape

(39, 16)

#### Create map for the clustered neighborhood

In [182]:
# create map
map_clusters = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [183]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Farmers Market,Restaurant
1,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Bank,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park
2,Downtown Toronto,0,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Café,Italian Restaurant,Japanese Restaurant,Tea Room,Bookstore
3,Downtown Toronto,0,Café,Coffee Shop,Cocktail Bar,American Restaurant,Gastropub,Creperie,Italian Restaurant,Restaurant,Clothing Store,Moroccan Restaurant
4,East Toronto,0,Trail,Health Food Store,Pub,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
5,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Restaurant,Cheese Shop,Café,Beer Bar,Japanese Restaurant,Hotel
6,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Japanese Restaurant,Sandwich Place,Café,Salad Place,Dessert Shop,Middle Eastern Restaurant,Thai Restaurant,Department Store
7,Downtown Toronto,0,Grocery Store,Café,Park,Restaurant,Baby Store,Italian Restaurant,Diner,Athletics & Sports,Coffee Shop,Nightclub
8,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Thai Restaurant,Hotel,Cosmetics Shop,American Restaurant,Bakery
9,West Toronto,0,Bakery,Pharmacy,Supermarket,Wine Shop,Grocery Store,Brewery,Park,Smoke Shop,Bar,Music Venue


In [184]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,1,Tennis Court,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [185]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,2,Park,Jewelry Store,Trail,Sushi Restaurant,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
33,Downtown Toronto,2,Park,Playground,Trail,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [186]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Park,Swim School,Bus Line,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [187]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Home Service,Music Venue,Garden,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
